In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 0

5


In [3]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=263,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='gin

In [4]:
df = pd.read_csv("../data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [7]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [8]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4033
4033
4033


In [9]:
display(df_train)
display(df_test)

,Unnamed: 0,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,...,B_Heavyweight_rank,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank
200,201,Darko Stosic,Kennedy Nzechukwu,-120,100,83.333333,100.000000,2019-08-03,"Newark, New Jersey, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither
201,202,Mickey Gall,Salim Touahri,100,-120,100.000000,83.333333,2019-08-03,"Newark, New Jersey, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither
202,203,Antonina Shevchenko,Lucie Pudilova,-160,140,62.500000,140.000000,2019-08-03,"Newark, New Jersey, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red
203,204,Jordan Espinosa,Matt Schnell,-155,145,64.516129,145.000000,2019-08-03,"Newark, New Jersey, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,Red
204,205,Lauren Murphy,Mara Romero Borella,180,-220,180.000000,45.454545,2019-08-03,"Newark, New Jersey, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4228,4301,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither
4229,4302,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither
4230,4303,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither
4231,4304,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither


,Unnamed: 0,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,...,B_Heavyweight_rank,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank
0,0,Jessica Eye,Cynthia Calvillo,120,-130,120.000000,76.923077,2020-06-13,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red
1,1,Karl Roberson,Marvin Vettori,210,-230,210.000000,43.478261,2020-06-13,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither
2,2,Charles Rosa,Kevin Aguilar,170,-185,170.000000,54.054054,2020-06-13,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither
3,3,Andre Fili,Charles Jourdain,-220,200,45.454545,200.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither
4,4,Jordan Espinosa,Mark De La Rosa,-167,157,59.880240,157.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,Colby Covington,Robbie Lawler,-300,250,33.333333,250.000000,2019-08-03,"Newark, New Jersey, USA",USA,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,Red
196,197,Jim Miller,Clay Guida,-140,130,71.428571,130.000000,2019-08-03,"Newark, New Jersey, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither
197,198,Joaquim Silva,Nasrat Haqparast,315,-380,315.000000,26.315789,2019-08-03,"Newark, New Jersey, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither
198,199,Trevin Giles,Gerald Meerschaert,-170,150,58.823529,150.000000,2019-08-03,"Newark, New Jersey, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither


In [10]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [11]:
models[1][model_num]

"LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)"

In [12]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [13]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [14]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena']

In [15]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_0
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=50000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=75, solver='lbfgs', tol=5.993308915320019,
                   verbose=0, warm_start=False)
['empty_arena', 'R_current_win_streak', 'title_bout', 'R_avg_SIG_STR_pct', 'longest_win_streak_dif', 'age_dif', 'B_win_by_Decision_Majority', 'B_win_by_TKO_Doctor_Stoppage', 'R_win_by_Decision_Majority', 'R_odds', 'R_current_lose_streak', 'R_Reach_cms', 'B_draw', 'total_round_dif', 'R_ev', 'B_ev']
0


In [16]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))

    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)

    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    
    
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
        
    

Current best score is: 16450.68030592314
Feature: B_odds Score: 16608.86874590233
Feature: B_wins Score: 16747.194523843835
Feature: R_draw Score: 17346.386324907922
Feature: R_total_rounds_fought Score: 17558.047767324297
Feature: R_total_title_bouts Score: 18055.973882942668
The best feature was R_total_title_bouts.  It scored 18055.973882942668
Current best score is: 18055.973882942668
Feature: B_longest_win_streak Score: 18312.986290635807
The best feature was B_longest_win_streak.  It scored 18312.986290635807
Current best score is: 18312.986290635807
Feature: R_losses Score: 18607.751533509592
The best feature was R_losses.  It scored 18607.751533509592
Current best score is: 18607.751533509592
Feature: R_longest_win_streak Score: 18987.71672586357
The best feature was R_longest_win_streak.  It scored 18987.71672586357
Current best score is: 18987.71672586357
NO IMPROVEMENT
FINAL BEST SCORE: 18987.71672586357


Starting New Run for LogisticRegression


Previous Best Score: 18987.

In [17]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3676, 20)

(3676,)

(3676, 2)

(200, 20)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,120,-130,0.439152,0.560848,1
1,210,-230,0.368521,0.631479,1
2,170,-185,0.452357,0.547643,0
3,-220,200,0.682194,0.317806,0
4,-167,157,0.663333,0.336667,0
...,...,...,...,...,...
195,-300,250,0.668939,0.331061,0
196,-140,130,0.460981,0.539019,0
197,315,-380,0.231809,0.768191,1
198,-170,150,0.641879,0.358121,1



          Number of matches: 200
          Number of bets: 134
          Number of winning bets: 65
          Number of losing bets: 69
          Number of underdog bets: 86
          Number of underdog wins: 33
          Number of underdog losses: 53
          Number of Favorite bets: 45
          Number of favorite wins: 31
          Number of favorite losses: 14
          Number of even bets: 3
          Number of even wins: 1
          Number of even losses: 2
          Profit: -589.3305892940928
          Profit per bet: -4.397989472343975
          Profit per match: -2.9466529464704636
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [18]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [19]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [20]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [21]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=263,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='gin